In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
dependent=dataset[['Purchased']]

In [13]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [14]:
independent.shape

(400, 3)

In [15]:
dependent.shape

(400, 1)

In [16]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [18]:
from sklearn.svm import SVC

In [21]:
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf', 'poly', 'sigmoid'],'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]}

grid=GridSearchCV(SVC(probability=True),param_grid,refit = True, verbose = 3,cv=5, n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   26.5s finished
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [22]:
re = grid.cv_results_
grid_prediction = grid.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(y_test,grid_prediction)

In [24]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [25]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("the f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

the f1_macro value for the best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9256751954513147


In [26]:
print ("the confusion Matrix:\n",cf)

the confusion Matrix:
 [[79  6]
 [ 4 45]]


In [27]:
print ("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9635054021608643

In [29]:
table=pd.DataFrame.from_dict(re)

In [30]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018604,0.004151,0.002354,0.002827,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
1,0.008194,0.001360,0.003228,0.001614,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.888889,0.850543,0.944161,0.941902,0.898278,0.038157,1
2,0.013166,0.002382,0.000805,0.000986,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.812243,0.766556,0.943699,0.899385,0.843805,0.066180,13
3,0.009046,0.002018,0.001208,0.000986,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.743410,0.655795,0.738451,0.801015,0.740137,0.047410,36
4,0.009456,0.002698,0.000889,0.001098,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
5,0.007765,0.002439,0.001487,0.002118,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.888889,0.850543,0.944161,0.941902,0.898278,0.038157,1
6,0.009769,0.002499,0.001500,0.001337,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.812243,0.766556,0.943699,0.899385,0.843805,0.066180,13
7,0.005510,0.001670,0.001284,0.001585,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.743410,0.655795,0.738451,0.801015,0.740137,0.047410,36
8,0.034646,0.002745,0.001204,0.000983,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.753889,0.665930,0.943041,0.899385,0.807042,0.100426,21
9,0.013705,0.003104,0.001203,0.000982,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.888889,0.832483,0.944161,0.941902,0.894680,0.043030,3
